The study of logic has two origins, philosophy and geometry. In many places throughout the world, people tried to establish rules of logic in order to clarify and resolve philosophical and political arguments.

However, in ancient Greece, there was a rather different context in which logic was studied. Over several centuries, the cultures of Greece, Babylon, Egypt, and more, all had discovered thousands of geometric ideas and theorems. For instance, it was known that

-   The interior angles of a triangle sum to 180 degrees.

-   The Pythagorean theorem, relating the side lengths of a right triangle.

-   For any line and point not on the line, there is a parallel line through the point.


In [1]:
from manim import *
config.media_width = "100%"

Manim Community v0.17.3

In [4]:
%%manim -qm -v WARNING LinePointParallel

class LinePointParallel(Scene):
    def construct(self):
        l = Line([7,9,0],[-5,-5,0])
        self.play(Create(l))
        p = Dot([2,-1,0])
        self.play(Create(p))
        self.wait(3)

In [7]:
import pyparsing
# Here we have to define a "parser", which can be a complicated topic.
# Rather than explain it, just test the following code to see what it 
# does.

terms = pyparsing.Word(pyparsing.alphas) | "not" | "and" | "or" | "if" | "then" | "iff" 
nesting = pyparsing.nestedExpr( '(', ')', content=terms)

# print(nesting.parseString("((not P) or (not Q)) or R"))
# print(nesting.parseString("(((not P) or (not Q)) or R)"))

# An abstract class
class Proposition:
    pass

class PropVariable(Proposition):
    def __init__(self, v):
        assert (type(v) == str and len(v) == 1) 
        self.v = v
    def __str__(self):
        return(self.v)

class Negation(Proposition):
    def __init__(self, beta):
        assert issubclass(type(beta),Proposition)
        self.neg = beta
    def __str__(self):
        return("(not " + str(self.neg) + ")")

class Disjunction(Proposition):
    def __init__(self, beta, gamma):
        assert issubclass(type(beta),Proposition) \
            and issubclass(type(gamma),Proposition)
        self.left, self.right = beta, gamma
    def __str__(self):
        return("("+str(self.left)+" or " + str(self.right) + ")")

p, q, r = PropVariable("P"), PropVariable("Q"), \
    PropVariable("R")

notp = Negation(p)
notporq = Disjunction(q,notp)

string = "(not P) or Q"
    
def parseTree(s): 
    p = nesting.parseString("("+s+")")[0]
    # First, there can be a lot of extraneous list embedding, especially 
    # if there were extraneous parentheses.  So we peel them off until 
    # there is something interesting.  
    while type(p) == type([]) and len(p) == 1:
        p = p[0]
    return pthelper(p)

def pthelper(p):
    if type(p) == type(""):
        return PropVariable(p)
    if len(p) == 2:
        assert( p[0] == "not" )
        return Negation(pthelper(p[1]))
    if len(p) == 3:
        if p[1] == "or": return Disjunction(pthelper(p[0]), pthelper(p[2]))
        raise Exception("length 3 list but unrecognized middle token")
    raise Exception("length of list unrecognized")

str(parseTree(string))

'((not P) or Q)'